# Inventário

Deve-se executar as células abaixo para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [4]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [5]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [6]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [7]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [8]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def get_sign_color(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [10]:
def get_sign_film(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.upper().split("TIPO ")[1].strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [11]:
def get_sign_approval(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [12]:
!cat .env

STAGE=PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [142]:
filename='rem_10'

Escolhendo a aba da planilha do Excel

In [143]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
ws = wb[sheetname]
ws

<Worksheet "Total_1-2023">

Testando planilha carregada do Excel

In [144]:
print(ws['A1'].value)

Código ARTESP


Carregando os valores no script

In [145]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

Teste de valores importados da planilha

In [146]:
len(values)

1014

Definindo valores da importação

In [147]:
company = Company.objects.get(name='Entrevias')

occurrence_type=OccurrenceType.objects.get(uuid='03f21bad-d751-47c2-8810-374f2f4f1259')
# occurrence_type_reporting=OccurrenceType.objects.get(uuid='2bffd7a4-3c7a-4ce8-b3b5-1b5a4943fa8f')
user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')

sign_name='Adicionar Retrorrefletância de película'
company,occurrence_type,user,status

(<Company: 91d69c92-2284-4570-9011-760a5a74a708: Entrevias>,
 <OccurrenceType: Sinalização vertical - ['ELO SP Vias Antiga (Não utilizada)', 'Entrevias']>,
 <User: rlcs - Kartado CS>,
 <ServiceOrderActionStatus: Identificado - REPORTING_STATUS>)

Criando características da placa

In [148]:
# for a in tqdm(values):
#     a['Películas'] = list(filter(None, [{'color': get_sign_color(sign_name, 'Cor', occurrence_type, a['cor 1']), 'type_of_film': get_sign_film(sign_name, 'Tipo de Película', occurrence_type, a['tipoPelicula 1'])},
#                       {'color': get_sign_color(sign_name, 'Cor', occurrence_type, a['cor 2']), 'type_of_film': get_sign_film(sign_name, 'Tipo de Película', occurrence_type, a['tipoPelicula 2'])} if a['cor 2'] != '' and a['cor 2'] is not None else None,
#                       {'color': get_sign_color(sign_name, 'Cor', occurrence_type, a['cor 3']), 'type_of_film': get_sign_film(sign_name, 'Tipo de Película', occurrence_type, a['tipoPelicula 3'])} if a['cor 3'] != '' and a['cor 3'] is not None else None,
#                       {'color': get_sign_color(sign_name, 'Cor', occurrence_type, a['cor 4']), 'type_of_film': get_sign_film(sign_name, 'Tipo de Película', occurrence_type, a['tipoPelicula 4'])} if a['cor 4'] != '' and a['cor 4'] is not None else None]))

Criando levantamentos de retrorrefletância

In [149]:
# for a in tqdm(values):
#     a['Retro'] = list(filter(None, [{'color': get_sign_color(sign_name, 'Cor', occurrence_type_reporting, a['cor 1']), 'type_of_film': get_sign_film(sign_name, 'Tipo de Película', occurrence_type_reporting, a['tipoPelicula 1']), 'first': a['m1_1'], 'second': a['m2_1'], 'third': a['m3_1'], 'fourth': a['m4_1'], 'fifth': a['m5_1'], 'item_approval': get_sign_approval(sign_name, 'Situação', occurrence_type_reporting, a['Sit_Placa_1']), 'cod_pel': a['codAutoPel 1']},
#                       {'color': get_sign_color(sign_name, 'Cor', occurrence_type_reporting, a['cor 2']), 'type_of_film': get_sign_film(sign_name, 'Tipo de Película', occurrence_type_reporting, a['tipoPelicula 2']), 'first': a['m1_2'], 'second': a['m2_2'], 'third': a['m3_2'], 'fourth': a['m4_2'], 'fifth': a['m5_2'], 'item_approval': get_sign_approval(sign_name, 'Situação', occurrence_type_reporting, a['Sit_Placa_2']), 'cod_pel': a['codAutoPel 2']} if a['cor 2'] != '' and a['cor 2'] is not None else None,
#                       {'color': get_sign_color(sign_name, 'Cor', occurrence_type_reporting, a['cor 3']), 'type_of_film': get_sign_film(sign_name, 'Tipo de Película', occurrence_type_reporting, a['tipoPelicula 3']), 'first': a['m1_3'], 'second': a['m2_3'], 'third': a['m3_3'], 'fourth': a['m4_3'], 'fifth': a['m5_3'], 'item_approval': get_sign_approval(sign_name, 'Situação', occurrence_type_reporting, a['Sit_Placa_3']), 'cod_pel': a['codAutoPel 3']} if a['cor 3'] != '' and a['cor 3'] is not None else None,
#                       {'color': get_sign_color(sign_name, 'Cor', occurrence_type_reporting, a['cor 4']), 'type_of_film': get_sign_film(sign_name, 'Tipo de Película', occurrence_type_reporting, a['tipoPelicula 4']), 'first': a['m1_4'], 'second': a['m2_4'], 'third': a['m3_4'], 'fourth': a['m4_4'], 'fifth': a['m5_4'], 'item_approval': get_sign_approval(sign_name, 'Situação', occurrence_type_reporting, a['Sit_Placa_4']), 'cod_pel': a['codAutoPel 4']} if a['cor 4'] != '' and a['cor 4'] is not None else None]))

Arrumando as fotos

In [150]:
for a in tqdm(values):
    if a['Foto_1'] is None:
        continue
    elif a['Foto_2'] is None and a['Foto_3'] is None and a['Foto_4'] is None and a['Foto_5'] is None:
        a['img']=list([a['Foto_1'].strip()])
    elif a['Foto_3'] is None and a['Foto_4'] is None and a['Foto_5'] is None:
         a['img']=list([a['Foto_1'].strip(), a['Foto_2'].strip()])
    elif a['Foto_4'] is None and a['Foto_5'] is None:
        a['img'] = a['img']=list([a['Foto_1'].strip(), a['Foto_2'].strip(), a['Foto_3'].strip()])
    elif a['Foto_5'] is None:
        a['img']=list([a['Foto_1'].strip(), a['Foto_2'].strip(), a['Foto_3'].strip(), a['Foto_4'].strip()])
    elif a['Foto_2'] is None:
        a['img']=list([a['Foto_1'].strip()])
    elif a['Foto_3'] is None:
        a['img']=list([a['Foto_1'].strip(), a['Foto_2'].strip()])
    elif a['Foto_4'] is None:
        a['img'] = a['img']=list([a['Foto_1'].strip(), a['Foto_2'].strip(), a['Foto_3'].strip()])
    else:
#         print(a['Foto_1'], a['Foto_2'], a['Foto_3'], a['Foto_4'], a['Foto_5'])
        a['img']=list([a['Foto_1'].strip(), a['Foto_2'].strip(), a['Foto_3'].strip(), a['Foto_4'].strip(), a['Foto_5'].strip()])

  0%|          | 0/1014 [00:00<?, ?it/s]

Testando valores modificados

In [151]:
v=values[0]
for a in values:
    if a['Nome da placa'] == 'M.A':
        a['Nome da placa'] = 'M.A.'

Criando itens de Inventário e Apontamentos

In [152]:
objects=[]
objects_reporting=[]

for index, a in enumerate(tqdm(values)):
    
    point, road = get_road_coordinates(a['Rodovia'].strip(), 
                                       float(str(a['km'])), 
                                             get_connected_reference(company, 
                                                                     'reporting', 
                                                                     'direction', 
                                                                     a['Sentido'].capitalize()) ,company)

    try:
        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=get_connected_reference(company, 'reporting', 'lane', a['Faixa'].capitalize()),
                    road=road,
                    road_name=road.name,
                    direction=get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize()),
                    created_by=user,
                    km=float(str(a['km'])),
                    end_km=float(a['km']),
                    firm=Firm.objects.get(company=company, name=a['Equipe']) if 'Equipe' in a else Firm.objects.get(company=company, name=a['Equipe/Empreiteira']),
                    found_at=a['Encontrado em'].replace(tzinfo=pytz.timezone('America/Sao_Paulo')),
                    point=point,
                    form_data={
                        'position': get_value('Posição', occurrence_type, a['Posição'].capitalize()) if a['Posição'] is  not None else None,
                        'sign_type': get_value('Tipo da placa', occurrence_type, a['Tipo da placa'].strip().title()) if a['Tipo da placa'] is not None else None,
                        'localization': get_value('Localização', occurrence_type, a['Localização'].capitalize()) if a['Localização'] is not None else None,
                        'side': get_value('Lado', occurrence_type, a['Lado'].strip().capitalize()) if a['Lado'] is not None else None,
                        'sign_name': get_value('Nome da placa', occurrence_type, a['Nome da placa'].strip().replace(" C", "/C")) if a['Nome da placa'] is not None else None,
                        'manufacture_date': a['Data de fabricação'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data de fabricação'] is not None and type(a['Data de fabricação']) is datetime.datetime else None,
                        'plate_type': get_value('Tipo de Chapa', occurrence_type, a['Tipo de Chapa'].strip().capitalize()) if a['Tipo de Chapa'] else None,
                        'shape': get_value('Formato', occurrence_type, a['Formato'].replace("Â", "A").capitalize()) if a['Formato'] is not None else None,
#                         'diameter': float(a['Diâmetro (m)']) if a['Diâmetro (m)'] is not None else None,
                        'width': float(a['Largura (m)']) if a['Largura (m)'] != 'CIRCULAR' else None,
                        'height': float(a['Altura (m)']) if a['Altura (m)'] != 'CIRCULAR' else None,
                        'notes': a['Observações'] if a['Observações'] is not None else None,
                        'artesp_number': str(a['Código ARTESP']) if a['Código ARTESP'] else None}
                    ), a['img']))


    except Exception as e:
    #             print(f"AQUI: {a['Rodovia']}, {a['km']} ou {a['Nome da Placa']}")
        print(f'LINHA:{index+2} | Rodovia:{a["Rodovia"]} km: {a["km"]}')
print(filename)

  0%|          | 0/1014 [00:00<?, ?it/s]

rem_10


Salvando itens de Inventário no sistema

In [153]:
print(filename)
for reporting, images in tqdm(objects):
    reporting.save()
#     print(reporting.number)

rem_10


  0%|          | 0/1014 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object ha

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 312: SPA-325/322
'NoneType' object ha

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 291: SPA-327/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 291: SPA-327/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 291: SPA-327/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 291: SPA-327/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 291: SPA-327/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 291: SPA-327/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 291: SPA-327/330
'NoneType' object ha

...

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 294: SPA-355/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 294: SPA-355/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 294: SPA-355/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 294: SPA-355/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 294: SPA-355/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 294: SPA-355/330
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 294: SPA-355/330
'NoneType' object ha

Searching KM on direction...
Searching KM without direction...
Found KM on Road 305: SPA-375/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 305: SPA-375/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 305: SPA-375/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 305: SPA-375/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 305: SPA-375/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 305: SPA-375/322
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 305: SPA-375/322
'NoneType' object has no attribute 'order'
Searching KM on dire

'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Sea

Searching KM on direction...
Searching KM without direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 314: SPA-468/328
'NoneType' object has no attribute 'order'
Searching KM on dire

'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 316: SPA-470/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 316: SPA-470/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 316: SPA-470/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 316: SPA-470/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 316: SPA-470/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 316: SPA-470/328
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 316: SPA-470/328
'NoneType' object ha

Anexando as imagens

In [154]:
images_bulk=[]

for reporting, images in tqdm(objects):
    for image in images:        
        images_bulk.append(ReportingFile(
            created_by=user,
            reporting=reporting,
            description="",
            upload='entrevias_inventario_2023/' + image,
            datetime=reporting.found_at if reporting.found_at else "",
            point=reporting.point,
            km=reporting.km))
        try:
            images_bulk[-1].upload.size
        except Exception as err:
            try:
                image=image.replace(".jpg", ".JPG")                
                images_bulk.pop()
                images_bulk.append(ReportingFile(
                    created_by=user,
                    reporting=reporting,
                    description="",
                    upload='entrevias_inventario_2023/' + image,
                    datetime=reporting.found_at if reporting.found_at else "",
                    point=reporting.point,
                    km=reporting.km))
                images_bulk[-1].upload.size
            except Exception as e:
                print(image)

  0%|          | 0/1014 [00:00<?, ?it/s]

In [155]:
photos=bulk_create_with_history(images_bulk, ReportingFile, batch_size=300)

In [ ]:
# rem_3: 
# Vertical_399+280_maiquel_EntreVias_20230306_075158.JPG
# Vertical_356+725_maiuel_EntreVias_20230413_094114.JPG
# rem_4:
# Vertical_317+000_egatibanco _EntreVias_20230406_130750
# Vertical_317+000_egatibanco _EntreVias_20230406_130830
# Vertical_365+700_maiuel_EntreVias_20230411_100205.JPG

# Final (linhas de teste abaixo)

In [63]:
import re
def convert_string(string: str) -> str:
    if '_' in string:
        # Convert snake_case to camelCase
        return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), string)
    else:
        # Convert camelCase to snake_case
        return re.sub(r'([A-Z])', lambda x: '_' + x.group(1).lower(), string)

In [65]:
for a in occurrence_type.form_fields['fields']:
    if a.get("apiName"):
        print(f'"{convert_string(a["apiName"])}": a["{a["displayName"]}"]{a["dataType"]} if a["{a["displayName"]}"] is not None else None,')
    else:
        print(f'"{convert_string(a["api_name"])}": a["{a["display_name"]}"]{a["dataType"]} if a["{a["displayName"]}"] is not None else None,')
    if a.get('dataType') == 'arrayOfObjects':
        for b in a['innerFields']:
            print(f'\t"{convert_string(b["apiName"])}": a["{b["displayName"]}"]{a["dataType"]} if a["{a["displayName"]}"] is not None else None,')
    if a.get('data_type') == 'arrayOfObjects':
        for b in a['inner_fields']:
            print(f'\t"{convert_string(b["api_name"])}": a["{b["display_name"]}"]{a["dataType"]} if a["{a["displayName"]}"] is not None else None,')

"new_ra": a["RA Novo"]float if a["RA Novo"] is not None else None,
"manufacture_date": a["Data de fabricação"]timestamp if a["Data de fabricação"] is not None else None,
"localization": a["Localização"]select if a["Localização"] is not None else None,
"sign_type": a["Tipo da placa"]select if a["Tipo da placa"] is not None else None,
"sign_name": a["Nome da placa"]select if a["Nome da placa"] is not None else None,
"plate_type": a["Tipo de Chapa"]select if a["Tipo de Chapa"] is not None else None,
"support_type": a["Tipo do Suporte"]select if a["Tipo do Suporte"] is not None else None,
"section": a["Perfil"]select if a["Perfil"] is not None else None,
"width": a["Largura (m)"]float if a["Largura (m)"] is not None else None,
"height": a["Altura (m)"]float if a["Altura (m)"] is not None else None,
"diameter": a["Diâmetro (m)"]float if a["Diâmetro (m)"] is not None else None,
"area_with_diameter": a["Area"]jsonLogic if a["Area"] is not None else None,
"support_dimension": a["Dimensão do su

In [132]:
for a in Road.objects.filter(company=company).order_by('name'):
    m=a.marks
    keys=list(m.keys())
    first_key = keys[0]
    last_key = keys[-1]
    first_item =  m[first_key]['km']
    last_item =  m[last_key]['km']
    print(f'Rodovia: {a.name} | km inicial:{first_item} > km final:{last_item}')

Rodovia: SP-266 | km inicial:486.492 > km final:509.036
Rodovia: SP-294 | km inicial:451.5 > km final:458
Rodovia: SP-322 | km inicial:307.5 > km final:391.2
Rodovia: SP-328 | km inicial:472.0 > km final:475.74
Rodovia: SP-328 | km inicial:468.39 > km final:472.0
Rodovia: SP-328 | km inicial:323.13 > km final:337.2
Rodovia: SP-330 | km inicial:318.5 > km final:450.11
Rodovia: SP-333 | km inicial:333 > km final:400.988
Rodovia: SP-333 | km inicial:444.519 > km final:450.99
Rodovia: SP-333 | km inicial:212.45 > km final:327.4
Rodovia: SP-333 | km inicial:404.25 > km final:440.8
Rodovia: SP-351 | km inicial:128.7 > km final:150.44
Rodovia: SPA-135/351 | km inicial:0.0 > km final:1.3
Rodovia: SPA-245/333 | km inicial:0.05 > km final:2.605
Rodovia: SPA-245/333 | km inicial:3.61 > km final:13.25
Rodovia: SPA-274/333 | km inicial:0.0 > km final:0.3
Rodovia: SPA-321/322 | km inicial:0.0 > km final:3.56
Rodovia: SPA-325/322 | km inicial:0.0 > km final:8.55
Rodovia: SPA-327/330 | km inicial:0.0 

In [126]:
m=Road.objects.filter(company=company)[0].marks
keys = list(m.keys())


(318.5, 450.11)